<a href="https://colab.research.google.com/github/kevgam/CAS_IE_Information_Retrieval/blob/main/IR_Test_NEU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vorbereitung des Datensatzes in Spark

Unser ursprünglicher Spotify-Datensatz, der über [Kaggle](https://www.kaggle.com/discussions/accomplishments/522912) verfügbar ist, umfasste fast eine Million Datensätze. Aufgrund der Grösse des Datensatzes hatten wir bei der Verarbeitung Performanceprobleme. Daher haben wir den Datensatz zunächst in der Spark-Umgebung der ZHAW vorverarbeitet.


Nach dem Standardlogin (inkl. sc.stop() am Schluss) gemäss Anleitung der ZHAW haben wir folgende Schritte ausgeführt:

### Installation und Test der Umgebung

```python
# Installation des notwendigen Pakets
sparky.installpackage('langdetect')

# Alternativ mit pip
pip install langdetect

# Test der RDD-Funktionalität
import os
liste = range(16)
rdd = sc.parallelize(liste)
print(rdd.collect())
print(rdd.glom().collect())

# Überprüfen, ob alle Worker die notwendige Software installiert haben
if len(list(filter(lambda x: x == [], rdd.glom().collect()))):
    raise SystemExit("Nicht gut - einige Worker bleiben ohne Softwareinstallation.")

# Testfunktion für Abhängigkeiten
def testdep(ignore_arg):
    ip = "160.85.252.66"  # Beispiel-IP
    try:
        import lxml
    except:
        return f"lxml FAILED! @ {ip}"
    else:
        return f"lxml worked @ {ip}"

# Installation von Abhängigkeiten
import subprocess
def installdeps(ignore_arg):
    p = subprocess.run("pip install lxml", shell=True, stdout=subprocess.PIPE)
    return p.stdout.decode()

# Ausführen der Installation und Tests
rdd.map(installdeps).collect()
rdd.map(testdep).collect()


### Laden und Verarbeiten der Daten

```python
# Das File wurde vorgängig in unseren Ordner auf dem Server kopiert
filepath = 'songs_with_attributes_and_lyrics.csv'

# Laden der CSV-Datei
import pandas as pd
dfs = pd.read_csv(filepath)

# Spark DataFrame laden
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
dfs = spark.read.csv(filepath, header=True, inferSchema=True)


### Sprache erkennen und Fortschritt protokollieren

```python
from langdetect import detect

# Funktion zur Erkennung der Sprache mit Fortschrittsanzeige
def detect_language_with_progress(partition):
    total_rows = 0
    for row in partition:
        try:
            lang = detect(row['lyrics'])
            yield (row['lyrics'], lang)  # Rückgabe: Originaltext und erkannte Sprache
        except Exception:
            yield (row['lyrics'], 'unknown')
        total_rows += 1
        if total_rows % 1000 == 0:  # Fortschritt alle 1000 Zeilen anzeigen
            print(f"Processed {total_rows} rows in this partition")

# RDD-Transformationen anwenden
rdd = dfs.rdd.mapPartitions(detect_language_with_progress)

# Zurück in ein DataFrame umwandeln
schema = StringType()
result = rdd.toDF(["lyrics", "lyrics_language"])

# Fortschritt anzeigen
result.show()


### Ergebnisse speichern

```python
# Als Excel-Datei speichern
output_path = './processed_songs_with_lyrics.xlsx'
dfs.to_excel(output_path, index=False)
print(f"DataFrame saved to: {output_path}")

# Als CSV-Datei speichern
csv_output_path = './processed_songs_with_lyrics.csv'
dfs.to_csv(csv_output_path, index=False)
print(f"DataFrame saved as CSV file to: {csv_output_path}")


### Filterung und Speicherung der englischen Texte

```python
# Zeilen filtern, in denen die Sprache Englisch ist
dfs_en = dfs[dfs['lyrics_language'] == 'en']

# Gefilterte Daten als Excel- und CSV-Datei speichern
dfs_en.to_excel('./processed_songs_filtered_lyrics_en.xlsx', index=False)
dfs_en.to_csv('./processed_songs_filtered_lyrics_en.csv', index=False)

print("Filtered DataFrame saved as 'filtered_lyrics_en.xlsx' and 'filtered_lyrics_en.csv'")


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Herausfiltern der Datensätze ohne Albumnamen

```python
# Installation pandas und openpyxl
!pip install pandas openpyxl

import pandas as pd

# Laden der CSV-Datei
file_path = "/content/drive/MyDrive/ie_information_retrieval_dataset/processed_songs_filtered_lyrics_en.csv"
df = pd.read_csv(file_path)

# Filtern aller Zeilen, bei denen 'album_name' nicht NaN ist
df_filtered = df.dropna(subset=['album_name'])

# Speichern der Datei als CSV
output_file_path = '/content/drive/MyDrive/ie_information_retrieval_dataset/processed_songs_filtered_lyrics_with_album_name.csv'
df_filtered.to_csv(output_file_path, index=False)


In [20]:
# Installation der benötigten Bibliotheken
!pip install pandas openpyxl

import pandas as pd
import re

# Lade die Excel-Datei
file_path = "/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_filtered_lyrics_en.xlsx"
df = pd.read_excel(file_path)

# Entferne die Spalten 'id' und 'album_name'
df = df.drop(columns=['id', 'album_name', 'lyrics_language'], errors='ignore')

# Entferne alle Datensätze, bei denen 'duration_ms' kleiner als 240000 (4 Minuten) oder größer als 300000 (5 Minuten) ist
df = df[(df['duration_ms'] >= 240000) & (df['duration_ms'] <= 300000)]

# Entferne Sonderzeichen in den Spalten 'name' und 'artists'
# Sonderzeichen inkl. [] werden entfernt
df['name'] = df['name'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))
df['artists'] = df['artists'].apply(lambda x: re.sub(r'[^\w\s]', '', str(x)))

# Filtere alle Zeilen, bei denen 'name' NaN ist
df = df.dropna(subset=['name'])

# Filtere alle Zeilen, bei denen 'artists' NaN ist
df = df.dropna(subset=['artists'])

# Entferne Duplikate basierend auf der Kombination aus 'name' und 'artists'
df = df.drop_duplicates(subset=['name', 'artists'])

# Sicherstellen, dass die Spalte 'lyrics' nur String-Werte enthält
df = df[df['lyrics'].apply(lambda x: isinstance(x, str))]

# Entferne Zeilenumbrüche innerhalb der 'lyrics' Spalte
df['lyrics'] = df['lyrics'].apply(lambda x: str(x).replace('\n', ' ').replace('\r', ' ') if isinstance(x, str) else x)

# Setze die 'lyrics'-Spalte in Anführungszeichen
df['lyrics'] = df['lyrics'].apply(lambda x: f'"{x}"' if isinstance(x, str) else x)

# Entferne Duplikate basierend auf der Kombination aus 'duration_ms' und 'lyrics'
df = df.drop_duplicates(subset=['duration_ms', 'lyrics'])

# Speichere die bereinigten Daten als CSV-Datei
output_csv_path = '/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_filtered_lyrics_bereinigt.csv'
df.to_csv(output_csv_path, index=False)

print(f"Bereinigte CSV-Datei wurde gespeichert unter: {output_csv_path}")


Bereinigte CSV-Datei wurde gespeichert unter: /content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_filtered_lyrics_bereinigt.csv


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import os

# NLTK-Resourcen herunterladen
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# DataFrame laden
df = pd.read_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_filtered_lyrics_bereinigt.csv')

# Umwandlung der gesamten 'lyrics' Spalte in Strings
df['lyrics'] = df['lyrics'].astype(str)

# Funktion zur Textbereinigung
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenisierung & Kleinschreibung
    tokens = [word for word in tokens if word.isalpha()]  # Sonderzeichen/Zahlen entfernen
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Stopwörter entfernen
    return " ".join(tokens)

# Neue Spalte "clean_lyrics" erstellen, ohne die Original-Lyrics zu überschreiben
df['clean_lyrics'] = df['lyrics'].apply(preprocess_text)

# Ergebnisse speichern
df.to_csv('/content/drive/MyDrive/ie_scripting_datasets/Archive/processed_songs_without_stopwords.csv', index=False)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
